In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import pymysql as ps
import time
import socket
import socks


class GooSpider:
    def __init__(self, query_sting, start=10):
        self.query_sting = query_sting
        self.headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
        }
        self.start = start

    def _get_response(self,url='https://scholar.google.com.hk/scholar'):
        
        proxies = {
                   'http' : 'http://127.0.0.1:8087',
                   'https': 'https://127.0.0.1:8087'
                  }
        

        query_generator = {
            "q": self.query_sting,
            "start": self.start
        }
        response = requests.get(url, headers=self.headers, params=query_generator)  # proxies=proxies
        print(response.status_code)
        html = response.text
        
        return html

    def get_articles(self):
        soup = BeautifulSoup(self._get_response(), "lxml")
        total_pages = len(soup.find_all('td'))-2
        articles = []
        for item in soup.find_all('div', class_='gs_r'):
            article_title = item.h3.get_text().replace('[HTML][HTML] ', '')
            ref_item = item.find('div', class_='gs_ggsd')
            article_ref = ref_item.a.get('href') if ref_item else 'null'
            citation = ''
            article_list = [datetime.now(), article_title, article_ref, self.query_sting, citation]
            articles.append(article_list)
        #print(articles)
        return articles, total_pages
    def get_pdf(self):
        #print(self.get_articles()[0][2])
        for i,li in enumerate(self.get_articles()[0]):            
            if li[2].endswith('.pdf'):
                try:
                    find_pdf = requests.get(li[2], headers=self.headers, verify = False).content
                    time.sleep(3)
                    pdf_name = li[2].split('/')[-1]
                    print(li[2])
                    root = 'D://'
                    path = root + pdf_name
                    time.sleep(10)
                    with open(path, 'wb') as f:
                        f.write(find_pdf)
                        print('PDF保存成功')
                        f.close()
                except:
                    print('有点问题,采取sci-hub试试')
#             else:
#                 print('尝试通过sci-hub下载，希望不要出现验证码')
#                 #try:
#                 url_to_sci = 'http://sci-hub.cc/'+li[1]
#                 find_pdf = requests.get(url_to_sci, headers=self.headers, verify = False).status_code #verify = False
#                 print(find_pdf)
# #                 except:
# #                     print('很遗憾，sci-hub也下不下来')

        print('文献下载完成')
        print(self.get_articles()[1])
            
        

def insert_to_db(articles):
    db_config = {
        'host': '10.10.172.41',
        'port': 3306,
        'user': 'xj',
        'password': 'xj3d',
        'db': 'dspdw_dev',
        'charset': 'utf8'
    }
    db = ps.connect(**db_config)
    cursor = db.cursor()
    print('数据库连接成功')
    insert_to_article = "INSERT INTO article VALUES(%s, %s, %s,%s, %s)"
    try:
        cursor.executemany(insert_to_article, articles)
        db.commit()
        print('数据导入成功')
    except:
        db.rollback()
        print('不知道哪儿出错')
    cursor.close()
    db.close()


        
        
    
    


goo = GooSpider('egfr')
articles = goo.get_articles()[0]

ConnectionError: HTTPSConnectionPool(host='scholar.google.com.hk', port=443): Max retries exceeded with url: /scholar?q=egfr&start=10 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000187B93FCF98>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))

In [ ]:
insert_to_db(articles)

In [17]:
def get_pdf_scihub(articles):
    headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
        }
    for i, article in enumerate(articles): 
        print(article)
        url_to_sci = 'http://sci-hub.cc/'+article[1]
        
        r = requests.get(url_to_sci, headers=headers)
        if r.headers['Content-Type'] == 'application/pdf':  
            root = 'D://'
            path = root + article[1] + '.pdf'
            with open(path, 'wb') as f:
                f.write(r.content)
                print('PDF保存成功')
                f.close()
            time.sleep(3)
#         else:
#             soup = BeautifulSoup(find_pdf_url,'lxml')
#             print(soup.find('iframe')['src'])

In [18]:
get_pdf_scihub(articles)

[datetime.datetime(2017, 9, 12, 17, 55, 36, 462070), 'Surface plasmon resonance scattering and absorption of anti-EGFR antibody conjugated gold nanoparticles in cancer diagnostics: applications in oral cancer', 'http://wiki.phy.queensu.ca/shughes/images/6/60/SPR_Scattering.pdf', 'egfr', '']
[datetime.datetime(2017, 9, 12, 17, 55, 36, 462070), 'Genotypic and histological evolution of lung cancers acquiring resistance to EGFR inhibitors', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3132801/', 'egfr', '']
[datetime.datetime(2017, 9, 12, 17, 55, 36, 463070), 'MET amplification occurs with or without T790M mutations in EGFR mutant lung tumors with acquired resistance to gefitinib or erlotinib', 'http://www.pnas.org/content/104/52/20932.full', 'egfr', '']
[datetime.datetime(2017, 9, 12, 17, 55, 36, 463070), 'Selective laser photo-thermal therapy of epithelial carcinoma using anti-EGFR antibody conjugated gold nanoparticles', 'https://www.researchgate.net/profile/Mostafa_El-Sayed/publicatio